In [1]:
# Imports
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.tree.export import export_text
from sklearn.ensemble import RandomForestClassifier 
from itertools import combinations_with_replacement
from itertools import permutations
from Bio.Seq import Seq

In [2]:
# file locations 
training_data = pd.read_csv('../rule-based-learning/datasets/humanSP1/humanSP1_train.csv', sep=',', header=None)
test_data = pd.read_csv('../rule-based-learning/datasets/humanSP1/humanSP1_test.csv', sep=',', header=None)
test_prediction = ('./humanSP1_predictions2.tsv')

In [3]:
#function to get all possible kmers of length k
def get_kmers(kmer_length): 

	all_kmers = []
	
	kmers = list(combinations_with_replacement("ATCG", kmer_length))
	for index in range(len(kmers)):
		kmers[index] = "".join(kmers[index])
		
	for kmer in kmers:
		permut = list(permutations(kmer))
		permut = list(set(["".join(x) for x in permut]))
		all_kmers = all_kmers + permut

	return all_kmers

#function to calculate the GC content of a list of sequences 
def calc_gc(sequences):
    
    gc_list = []
    
    for sequence in sequences:
    
        num_gc = 0
        for base in sequence:
            if base == "G" or base == "C":
                num_gc += 1

        gc_content = num_gc/len(sequence)
        gc_list.append(gc_content)

    return gc_list 

#function to get the list of kmer features for each sequences 
def get_features(data, kmer_list):

	features = []

	seq_len = len(data[0])
	kmer_length = len(kmer_list[0])

	for entry in data:

		feature_count = [0] * len(kmer_list) * 2
		start = 0
		end = kmer_length

		while end <= seq_len + 1:
			seq_slice = entry[start:end]
			start += 1
			end += 1

			if len(seq_slice) == kmer_length:
				#append to list of counts
				index = kmer_list.index(seq_slice)
				feature_count[index] += 1 

		#get the gc content and add this to the list of features for this sequence  
		#gc_content = calc_gc(entry)
        
        #feature_count.append(gc_content)
        #append to list of all features 
		features.append(feature_count)
		
	return features

#function to get the rev. complement of a list of sequences
def rev_comp(seqs):
    
    rev_seqs = []
    
    for seq in seqs:
        seq = Seq(seq)
        rev_seqs.append(seq.reverse_complement())
        
    return rev_seqs 

def combine_features(*args):
    
    combined_features = []
    
    num_features = len(args[0])
    
    for index in range(num_features):
        
        new_list = []
        
        for a in args:
            if isinstance(a[index], list):
                new_list = new_list + a[index]
            else:
                new_list = new_list + [a[index]]
        
        combined_features.append(new_list)
        
    return combined_features 

In [ ]:
# training data values (X = sequences and Y = TF classification)
X = training_data.values[:,0]
Y = training_data.values[:, 1:2]

fwd_seqs = X
rev_seqs = rev_comp(X)
gc_content = calc_gc(fwd_seqs)

for i in range(2,15):
    kmers = get_kmers(i)

    features_fwd = get_features(fwd_seqs, kmers)
    features_rev = get_features(rev_seqs, kmers)

    all_features = combine_features(features_fwd, features_rev)
        
    X_train, X_test, y_train, y_test = train_test_split(features_fwd, Y, test_size = 0.3, random_state = 100)

    tfbs_classifier = RandomForestClassifier(n_estimators=100, n_jobs=12, random_state=99)
    tfbs_classifier = tfbs_classifier.fit(X_train, np.ravel(y_train))
    y_pred = tfbs_classifier.predict(X_test)
    print("Number of kmers " + str(i))
    print("F1 score (fwd kmers only): " + str(f1_score(y_test, y_pred, average='macro')))
    
    X_train, X_test, y_train, y_test = train_test_split(all_features, Y, test_size = 0.3, random_state = 100)

    tfbs_classifier = RandomForestClassifier(n_estimators=100, n_jobs=12, random_state=99)
    tfbs_classifier = tfbs_classifier.fit(X_train, np.ravel(y_train))
    y_pred = tfbs_classifier.predict(X_test)
    print("F1 score (fwd and rev kmers): " + str(f1_score(y_test, y_pred, average='macro')))
    
    
#features_test = get_features(test_data.values[:,0], kmers)
#test_pred = tfbs_classifier.predict(features_test)

#with open(test_prediction, "w+") as out:
#	for entry in test_pred:
#		out.write(entry + "\n")    
    

Number of kmers 2
F1 score (fwd kmers only): 0.8457020729439975
F1 score (fwd and rev kmers): 0.8601976186646612
Number of kmers 3
F1 score (fwd kmers only): 0.849906191369606
F1 score (fwd and rev kmers): 0.8498722871887544
Number of kmers 4
F1 score (fwd kmers only): 0.8684982975226019
F1 score (fwd and rev kmers): 0.8602802215705442
Number of kmers 5
F1 score (fwd kmers only): 0.8582939705473743
F1 score (fwd and rev kmers): 0.8541261461517089
Number of kmers 6
F1 score (fwd kmers only): 0.8174178795814953
F1 score (fwd and rev kmers): 0.8129392446633825
